### 添加数据集和包文件

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"  # 指定使用 GPU 3

In [4]:
import os
import copy

import torch
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, EvalPrediction
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay

In [6]:
from transformers import AutoModel, AutoTokenizer

# 加载模型
model_0 = AutoModelForSequenceClassification.from_pretrained("/home/mcq2/GitHub/aes2/topic-classificatio/output/fold_0/checkpoint-812")
model_1 = AutoModelForSequenceClassification.from_pretrained("/home/mcq2/GitHub/aes2/topic-classificatio/output/fold_1/checkpoint-812")


# 加载分词器
tokenizer = AutoTokenizer.from_pretrained("/home/mcq2/GitHub/aes2/topic-classificatio/output/fold_0/checkpoint-812")

## 处理数据

In [7]:
PATH = "/home/mcq2/GitHub/aes2/kaggle/input/learning-agency-lab-automated-essay-scoring-2/"


train_df = pd.read_csv(PATH + "train.csv")
#data1 = pd.read_csv(PATHS.train_path)
# persuade = pd.read_csv('/home/mcq2/GitHub/aes2/kaggle/input/additional_dataset/persuade_2.0_human_scores_demo_id_github.csv')

In [ ]:
# intersection = pd.merge(train_df, persuade, on="full_text", how="inner")[["essay_id", "full_text", "score", "prompt_name"]].reset_index(drop=True)
# difference = train_df[~train_df["essay_id"].isin(intersection["essay_id"])].reset_index(drop=True)
# intersection["score_and_prompt"] = intersection["score"].astype(str) + "-" + intersection["prompt_name"]
#intersection.head()
# ## 将insertion 分为了两个部分，并且增加了一列在intersection里面
# intersection["fold"] = None
# splitter = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
# folds = list(splitter.split(X=np.zeros(len(intersection)), y=intersection["score_and_prompt"].values))
# for fold_idx, (_, val_idx) in enumerate(folds):
#     intersection.loc[val_idx, "fold"] = fold_idx

In [5]:
# intersection["label"] =  intersection["prompt_name"].astype("category").cat.codes
# ## 建立一个pd, prompt_to_id， 内容为关键词和它对应的数值
# prompt_to_id = intersection.drop_duplicates(subset=("prompt_name", "label"))[["prompt_name", "label"]]
# label2id = {row["prompt_name"]: row["label"] for _, row in prompt_to_id.iterrows()}
# id2label = {row["label"]: row["prompt_name"] for _, row in prompt_to_id.iterrows()}

NameError: name 'prompt_to_id' is not defined

In [8]:
#intersection_ds = Dataset.from_pandas(intersection)
diff_ds = Dataset.from_pandas(train_df)
# intersection_ds = intersection_ds.map(lambda i: tokenizer(i["full_text"], max_length=1024, truncation=True), batched=True)
diff_ds = diff_ds.map(lambda i: tokenizer(i["full_text"], max_length=1024, truncation=True), batched=True)

Map:   0%|          | 0/17307 [00:00<?, ? examples/s]

### 添加模型文件

In [9]:
def compute_metrics(eval_pred: EvalPrediction) -> dict[str, float]:
    predictions = eval_pred.predictions
    y_true = eval_pred.label_ids
    y_pred = predictions.argmax(-1)
    acc = accuracy_score(y_true, y_pred)
    return {"acc": acc}

args = TrainingArguments(
    output_dir="output",
    report_to="none",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=8,
    learning_rate=1e-5,
    lr_scheduler_type="constant",
    warmup_ratio=0.0,
    num_train_epochs=1,
    weight_decay=0.01,
    optim="adamw_torch",
    fp16=torch.cuda.is_available()
)

/home/mcq2/anaconda3/envs/aes/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
eval_ds = diff_ds
train_ds = diff_ds

In [11]:
trainer_0 = Trainer(
        args=args, 
        model=model_0,
        train_dataset=train_ds, 
        eval_dataset=eval_ds, 
        tokenizer=tokenizer, 
        compute_metrics=compute_metrics
    )

trainer_1 = Trainer(
        args=args, 
        model=model_1,
        train_dataset=train_ds, 
        eval_dataset=eval_ds, 
        tokenizer=tokenizer, 
        compute_metrics=compute_metrics
    )


### 预测topic分类

In [10]:
test_preds_0 = trainer_0.predict(diff_ds)
test_preds_1 = trainer_1.predict(diff_ds)

In [11]:
predictions = []
predictions.append(test_preds_0.predictions)
predictions.append(test_preds_1.predictions)

In [12]:
predictions = np.stack(predictions, axis=0).mean(axis=0)# 平均两个fold的预测值

In [13]:
predictions = predictions.argmax(-1)

In [14]:
essay_id = diff_ds["essay_id"]
  # convert prompt id back to prompt name
result_df = pd.DataFrame({"essay_id": essay_id, "topic":predictions })
result_df

,essay_id,topic
0,000d118,1
1,000fe60,6
2,001ab80,3
3,001bdc0,4
4,002ba53,2
...,...,...
17302,ffd378d,4
17303,ffddf1f,5
17304,fff016d,0
17305,fffb49b,4
